In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings("ignore")

In [ ]:
base_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'valid')
diseases = os.listdir(train_dir)

In [ ]:
print(diseases)


In [ ]:
unique_plants = []
Not_diseased = 0
diseased = 0
for plant in diseases:
    if plant.split('___')[0] not in unique_plants:
        unique_plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        diseased +=1

In [ ]:
print(f"Unique plants : {unique_plants}\n")
print(f"Total No Unique plants : {len(unique_plants)}\n")
print(f"Total No of Diseases : {diseased}")

In [ ]:
disease_dir = {}
for disease in diseases:
    disease_dir[disease] = len(os.listdir(train_dir + '/' + disease))

In [ ]:
df = pd.DataFrame(index = disease_dir.keys(),data = disease_dir.values(),columns=["total_images"])
df.head()

In [ ]:
import seaborn as sns
df.sort_values(by='total_images',ascending=False,inplace=True)
df_top = df.head(10)
df_bottom = df.tail(10)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
sns.barplot(data=df_top,x=df_top.total_images,y=df_top.index,color = sns.xkcd_rgb["slate blue"])
plt.title(f"10 Plant-Disease Classes With Most Images",fontweight="bold",fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(data=df_bottom,x=df_bottom.total_images,y=df_bottom.index,color = sns.xkcd_rgb["cloudy blue"])
plt.title(f"10 Plant-Disease Classes with Least Images",fontweight="bold",fontsize=12)
plt.show()

In [ ]:
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = [224, 224]

In [ ]:
from glob import glob
glob_dir = os.path.join(train_dir,'*')
folders = glob(glob_dir)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from glob import glob



inception = InceptionV3(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

for layer in inception.layers:
    layer.trainable = False
    

x = Flatten()(inception.output)

prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Augmentation



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)



In [ ]:
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model.save('plantsmodel.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
img=image.load_img('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Black_rot/00e909aa-e3ae-4558-9961-336bb0f35db3___JR_FrgE.S 8593.JPG',target_size=(224,224))

x = image.img_to_array(img)

## Here we are scaling the image input
x=x/255
x = np.expand_dims(x, axis=0)



preds = model.predict(x)

In [ ]:
preds

In [ ]:
np.argmax(preds)